<a href="https://colab.research.google.com/github/sunshineluyao/ML-minihackthon-tutorial/blob/main/RL_GameDevelopment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Install the Packages and Set up the RL Env

To start, ensure all necessary libraries are installed. OpenAI Gym provides the RL environments, and W&B offers live monitoring.


In [ ]:
!pip install gym gymnasium shimmy stable-baselines3

# Step 2 Set up Real Time Monitoring

Log in to [W&B](https://docs.wandb.ai/quickstart/) using your API key for experiment tracking.


In [ ]:
import wandb
wandb.login()


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

We'll use the CartPole environment from OpenAI Gym. This is a simple environment suitable for demonstrating RL training.


# Step 3 Train the Model

In [ ]:
import gym
env = gym.make('CartPole-v1')


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


We'll use Stable-Baselines3's `PPO` algorithm for training.


In [ ]:
import gymnasium as gym  # Gymnasium instead of Gym
from stable_baselines3 import PPO

# Example: CartPole-v1 environment
env = gym.make('CartPole-v1')
model = PPO('MlpPolicy', env, verbose=1)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


Integrate W&B to log metrics during the training process.


In [ ]:
from wandb.integration.sb3 import WandbCallback

wandb.init(project="RL-with-Monitoring", sync_tensorboard=True)

# Train the model with W&B callback for monitoring
model.learn(total_timesteps=10000, callback=WandbCallback())


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 22.5     |
|    ep_rew_mean     | 22.5     |
| time/              |          |
|    fps             | 432      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 26.9        |
|    ep_rew_mean          | 26.9        |
| time/                   |             |
|    fps                  | 450         |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008647941 |
|    clip_fraction        | 0.102       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.000213    |
|    learning_rate        | 0.

Test the trained model, visualize the agent's performance, and produce a video.


# Step 4: Test the Model

In [ ]:
import gymnasium as gym
from gym.wrappers import RecordVideo
import os

# Create the environment with video recording capability
env = gym.make('CartPole-v1', render_mode="rgb_array")
env = RecordVideo(env, video_folder="./videos", episode_trigger=lambda episode_id: True)

# Ensure video folder exists
os.makedirs("./videos", exist_ok=True)

# Reset the environment
obs, info = env.reset()

# Run the episode and record video
for step in range(500):  # Run for a fixed number of steps
    # Get the action from the trained model
    action, _ = model.predict(obs)

    # Step the environment
    result = env.step(action)

    # Unpack the result correctly
    if len(result) == 4:  # Old step API (backward compatibility)
        obs, reward, done, info = result
        terminated, truncated = done, False  # Assume `done` is terminated
    else:  # New step API
        obs, reward, terminated, truncated, info = result

    # Stop if the episode is terminated or truncated
    if terminated or truncated:
        obs, info = env.reset()

# Close the environment
env.close()

# List recorded videos to debug
video_files = os.listdir("./videos")
print("Recorded Videos:", video_files)


/usr/local/lib/python3.10/dist-packages/gym/wrappers/record_video.py:78: UserWarning: WARN: Overwriting existing videos at /content/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Recorded Videos: ['rl-video-episode-3.meta.json', 'rl-video-episode-1.meta.json', 'rl-video-episode-1.mp4', 'rl-video-episode-2.meta.json', 'rl-video-episode-3.mp4', 'rl-video-episode-0.mp4', 'rl-video-episode-4.meta.json', 'rl-video-episode-4.mp4', 'rl-video-episode-2.mp4', 'rl-video-episode-0.meta.json']
